In [3]:
import pandas as pd
import numpy as np
import glob

In [83]:
ROSTERS_PATH = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\weekly_roster_processed'
CURRENT_ROSTER = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\weekly_roster_processed\\Apr_2021.xlsx'

df_rosters = pd.concat(map(pd.read_excel, glob.glob(f'{ROSTERS_PATH}\\*.xlsx')))
df_rosters.Primary_truck = df_rosters.Primary_truck.str.replace(' ','')

df_curr_roster = pd.read_excel(CURRENT_ROSTER)
df_curr_roster.Primary_truck = df_curr_roster.Primary_truck.str.replace(' ','')

current_runs = df_curr_roster.Primary_route.unique()
current_trucks = df_curr_roster.Primary_truck.unique()

df_rosters = df_rosters[df_rosters.Primary_route.isin(current_runs) & df_rosters.Primary_truck.isin(current_trucks)]     


occurrence = df_rosters.groupby(['Primary_truck', 'Primary_route'])['Run_type'].count().reset_index()     
total = df_rosters.groupby(['Primary_truck'])['Run_type'].count().reset_index()


table_ratio = pd.merge(occurrence, total, on='Primary_truck', how='left')

table_ratio['portion'] = table_ratio.Run_type_x /  table_ratio.Run_type_y

table_ratio = table_ratio[['Primary_truck','Primary_route', 'portion']]


In [144]:
MVEXP_PATH = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\all_other_mv_expense\\to_be_process\\Apr_2021.xlsx'   

df_mvexp = pd.read_excel(MVEXP_PATH)
# df_mvexp
# .groupby('Primary_truck')
# .groupby('Primary_truck')['portion'].sum()
# .groupby('Primary_route')['portion'].sum()
# table_ratio
# df_mvexp['Job No.']

df_merge_1 = pd.merge(df_mvexp, table_ratio, left_on='Job No.', right_on='Primary_truck')
# df_merge['amount'] = df_merge.portion * df_merge.Debit
# df_merge = df_merge[['catID', 'Primary_truck', 'Primary_route', 'amount']]

In [141]:
COMPLETE_PATH = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\all_other_mv_expense\\processed\\Apr_2021.xlsx'    
df_merge = df_merge.groupby(['catID','Primary_route'])['amount'].sum().reset_index().sort_values('catID')    
df_merge.to_excel(COMPLETE_PATH)
# df_merge

In [142]:
# .groupby()

In [148]:
df_merge_truks = df_merge_1.Primary_truck.unique()
np.setdiff1d(df_merge_truks, current_trucks)

array([], dtype=object)

In [1]:
import pandas as pd
import numpy as np
import xlwings as xw
import glob 

# From Excel sheet
# Start path
# D:\Run Analysis\BLOB_STORAGE\expenses_truck\all_other_mv_expense\to_be_process

# Complete path
# D:\Run Analysis\BLOB_STORAGE\expenses_truck\all_other_mv_expense\processed


# Use Ratio base to allocate MV expense that has number plate

# Aggregate all past roster to summary a list of Route Occurence for Trucks

CURRENT_WEEK = '18th_2021'


PATH_CURR_W_MVEXP = f'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\all_other_mv_expense\\to_be_process\\{CURRENT_WEEK}.xlsx'

# Roster before 6/Apr/2021
PATH_PAST_ROSTER = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\Rearranged_past_roster\\past_ros.csv'

# Roster after 6/Apr/2021
PATH_CURR_ROSTER = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\weekly_roster_processed'


# Roster Arrangement 
# Item Ratio 
# ======================================================================
# Calculate the MV expense Ratio for Items label with number plate 
# Past Roster Cleaning
df_past_ros = pd.read_csv(PATH_PAST_ROSTER)

# remove any appended time and select data before 6/Apr/2021
df_past_ros.Date = df_past_ros.Date.str.split(' ', 1 ,expand=True)[0]
df_past_ros.Date = pd.to_datetime(df_past_ros.Date, format='%d/%m/%Y')
df_past_ros.Primary_route = df_past_ros.Primary_route.str.replace(' ', '')


df_past_ros = df_past_ros.loc[df_past_ros.Date <= '20210406' ]
try:
    df_cur_ros = pd.concat(map(pd.read_excel, glob.glob(f'{PATH_CURR_ROSTER}//*.xlsx')))
    df_cur_ros.Primary_truck = df_cur_ros.Primary_truck.str.replace(' ', '')

    df_all_ros = pd.concat([df_past_ros, df_cur_ros])

except:

    df_all_ros = df_past_ros 
    
    # pd.concat([df_past_ros, df_cur_ros])


occurence = df_all_ros.groupby(['Primary_truck','Primary_route'])['Run_type'].count().reset_index()
total = df_all_ros.groupby('Primary_truck')['Run_type'].count().reset_index()

df_merge = pd.merge(occurence, total, how='left', on='Primary_truck')
df_merge['ratio'] = df_merge['Run_type_x'] / df_merge['Run_type_y']

mv_exp_ratio_table = df_merge[['Primary_truck','Primary_route','ratio']].sort_values('Primary_route')
# If Items label with FLEET, I will allocate it according to the portion of total income.

# =========================================================================

# Current Week MV-Expense
# df_mvexp = pd.read_excel(PATH_CURR_W_MVEXP)

# df_mvexp['Date'] = pd.to_datetime(df_mvexp['Date'], format="%d/%m/%Y")

# mv_exp_table = df_mvexp.groupby(['catID','Job No.'])['Debit'].sum().reset_index()


# df_merge1 = pd.merge(mv_exp_table, mv_exp_ratio_table, how='left', left_on='Job No.',right_on='Primary_truck')

# df_merge1['amount_portion'] = df_merge1['Debit'] * df_merge1['ratio']

# df_merge1_summary  = df_merge1.groupby(['catID','Primary_route'])['amount_portion'].sum().reset_index().sort_values('catID')


# # Display Ratio

# wb = xw.Book()

# wb.sheets.add()

# wb.sheets[0].name = 'Mv_exp_alloc_details'

# wb.sheets[0].range('A1').value = df_merge1


# wb.sheets[1].name = 'MV_Exp_ratio'

# wb.sheets[1].range('A1').value = mv_exp_ratio_table


# wb.sheets[2].name = 'MV_exp'

# wb.sheets[2].range('A1').value = df_mvexp

# wb.sheets[3].name = 'MV_exp_alloc_sum'

# wb.sheets[3].range('A1').value = df_merge1_summary

# # D:\Run Analysis\BLOB_STORAGE\expenses_truck\all_other_mv_expense\processed

# COMPLETE_PATH = f'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\all_other_mv_expense\\processed\\{CURRENT_WEEK}.xlsx'

# wb.save(COMPLETE_PATH)

# wb.close()




mv_exp_ratio_table

,Primary_truck,Primary_route,ratio
323,CQ 64 LO,BH,0.002786
406,WHITE UTE,BINAUDIT,1.000000
1,AB 40 PW,BINWEIGHING,0.015152
425,XN 10 HU,BINWEIGHING,0.002548
324,CQ 64 LO,BLUEHILLSETC,0.002786
...,...,...,...
91,BY 46 TO,VISY-NIWPR2,0.003817
224,CG 94 KY,VISY-NIWPR2/CBK,0.001639
113,CC 23 RU,WEIGH&RL1,0.007143
114,CC 23 RU,WEIGHING,0.007143
